In [ ]:
#!pip install pyspark

In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
.master("local[4]") \
.appName("Classification with iris") \
.config("spark.driver.memory","2g") \
.config("spark.executor.memory","4g") \
.getOrCreate()

In [5]:
df = spark.read.format("csv") \
.option("header",True) \
.option("inferSchema", True) \
.option("sep",",") \
.load("/content/iris.csv")

In [6]:
df.limit(5).toPandas().head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [7]:
df.describe().toPandas().head()

,summary,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,count,150,150,150,150,150
1,mean,5.843333333333335,3.0540000000000007,3.7586666666666693,1.1986666666666672,None
2,stddev,0.8280661279778637,0.43359431136217375,1.764420419952262,0.7631607417008414,None
3,min,4.3,2.0,1.0,0.1,Iris-setosa
4,max,7.9,4.4,6.9,2.5,Iris-virginica


In [11]:
import pyspark.sql.functions as F
df.groupBy("Species").agg(F.count("*").alias("sayi")).show()

+---------------+----+
|        Species|sayi|
+---------------+----+
| Iris-virginica|  50|
|    Iris-setosa|  50|
|Iris-versicolor|  50|
+---------------+----+



In [13]:
df.printSchema()

root
 |-- SepalLengthCm: double (nullable = true)
 |-- SepalWidthCm: double (nullable = true)
 |-- PetalLengthCm: double (nullable = true)
 |-- PetalWidthCm: double (nullable = true)
 |-- Species: string (nullable = true)



In [9]:
from pyspark.ml.feature import StringIndexer

label_df = df
output_indexer = StringIndexer(inputCol="Species", outputCol="label", handleInvalid="skip")
label_df = output_indexer.fit(label_df).transform(label_df)

In [29]:
label_df.limit(5).toPandas().sample(5)

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species,label
3,4.6,3.1,1.5,0.2,Iris-setosa,0.0
4,5.0,3.6,1.4,0.2,Iris-setosa,0.0
1,4.9,3.0,1.4,0.2,Iris-setosa,0.0
0,5.1,3.5,1.4,0.2,Iris-setosa,0.0
2,4.7,3.2,1.3,0.2,Iris-setosa,0.0


In [14]:
df.columns

['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm', 'Species']

In [15]:
from pyspark.ml.feature import VectorAssembler
assembler_df = label_df
assembler = VectorAssembler(inputCols=['SepalLengthCm', 'SepalWidthCm', 
                                       'PetalLengthCm', 'PetalWidthCm'], 
                            outputCol="features")
assembler_df = assembler.transform(assembler_df)

In [16]:
assembler_df.limit(5).toPandas().head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species,label,features
0,5.1,3.5,1.4,0.2,Iris-setosa,0.0,"[5.1, 3.5, 1.4, 0.2]"
1,4.9,3.0,1.4,0.2,Iris-setosa,0.0,"[4.9, 3.0, 1.4, 0.2]"
2,4.7,3.2,1.3,0.2,Iris-setosa,0.0,"[4.7, 3.2, 1.3, 0.2]"
3,4.6,3.1,1.5,0.2,Iris-setosa,0.0,"[4.6, 3.1, 1.5, 0.2]"
4,5.0,3.6,1.4,0.2,Iris-setosa,0.0,"[5.0, 3.6, 1.4, 0.2]"


### **🔍Model**

In [17]:
train_df, test_df = assembler_df.randomSplit([0.8, 0.2], seed=142)

In [18]:
from pyspark.ml.classification import LogisticRegression
logreg_obj = LogisticRegression(featuresCol="features", labelCol="label")
logreg_model = logreg_obj.fit(train_df)

In [19]:
test_result_df = logreg_model.transform(test_df)

In [21]:
test_result_df.select("label","prediction","probability").limit(5).toPandas().head()

,label,prediction,probability
0,0.0,0.0,"[1.0, 1.509579890352493e-46, 7.636130610363657..."
1,0.0,0.0,"[1.0, 2.0524922096428498e-53, 1.00374572148616..."
2,0.0,0.0,"[1.0, 8.960646440597879e-34, 1.438044951853352..."
3,0.0,0.0,"[1.0, 9.83098456769958e-37, 6.462859081567191e..."
4,0.0,0.0,"[0.9999999999999951, 4.792985143694644e-15, 1...."


In [20]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
accuracy = evaluator.evaluate(test_result_df)
print(accuracy)

0.8888888888888888
